# Stock Price Prediction 

In [100]:
(solving the wrong problem!) MARCOS FETURES

SyntaxError: invalid syntax (<ipython-input-100-9a58aa2f05fd>, line 1)

# I) Description of the problem

### On the problem.

* The problem of stock prediction concerns of being able to forecast a *Weakly Non stationary Time Series* so one can **sell** an active at a higher **Ask Price** that is was it's **Bid price** (buying at a smaller price than it was previously sold).


### On the properties of the data.

* Stocks, ETF's, hedge funds, can be modeled instantaneously as two Weakly Non stationary Time Series, one for *bid* prices and one for *ask* prices. By weakly Non stationary Time Series ($X(t)$), it is meant:

<center>First momentum invariance.</center>
$$E[X(t)] = E[X(t + \epsilon)]$$ <br/>

<center>Second momentum invariance.</center>
$$C(X(t)) = C(X(t + \alpha)) = E[(X(t) - E[X(t)])(X(t + \epsilon) - E[X(t + \epsilon)])]$$ <br/>
<center>Bounded energy.</center>
$$E[|X(t)|^2] < \infty $$ <br/>




* So, one should interpret this data as not predictable for longer distances but, among several, one can depicts a few techniques to be able to predict those series. Namely:

    - Short term predictions: It transforms a non-stationary series in a locally stationary series. Thus, predictable.
    - Using the first order derivative ($r(t) = (x(t) - x(t - 1))$): It erases the 'memory' of a non-stationary series. This is also called **RETURNS** of a financial time series.


### On the financial data.

* **Bid's** and **Ask's** are updated at every new *order* in the *order book*, this is called *tick*. For visual chartists, analyzing this *tick* data is too cumbersome, so one of **many** ways of visualizing it, is in the form of **Open, Close, High, Low**, henceforth, nicknamed after *OCHL*. Those are defined as:

![title](assets/ochl.png)


* Among those who do trade stocks, we can find the ones so called *Scalpers* and the *Swing traders*, where, respectively, one places its buy/sell orders in the same day whereas the other waits for several days. We will be dealing with the former.

# II) Loading data

In [96]:
import requests
import quandl
import os

import pandas as pd
import numpy as np

In [97]:
api_key = os.environ['QUANDL_API_KEY']
metadata_filename = 'WIKI_metadata.csv'
dataset_filename = 'WIKI_dataset.csv'

## Downloading and extracting the dataset

In [98]:
import zipfile
def get_data(url, filename, extract=True):
    r = requests.get(url, allow_redirects=True)
    filename = filename + '.zip' if extract else filename
    open(filename, 'wb').write(r.content)
    if not extract: return
    zipfile.ZipFile(filename, 'r').extractall('./')

In [99]:
get_data('https://www.quandl.com/api/v3/databases/WIKI/metadata?api_key=%s' % api_key, 
         metadata_filename)

get_data('https://www.quandl.com/api/v3/datatables/WIKI/PRICES/delta.json?api_key=%s' % api_key, 
         dataset_filename, extract=False)

In [70]:
metadata_df = pd.read_csv(metadata_filename)
dataset_df = pd.read_csv(dataset_filename)

In [71]:
metadata_df.head(20)

,code,name,description,refreshed_at,from_date,to_date
0,A,"Agilent Technologies Inc. (A) Prices, Dividend...","End of day open, high, low, close and volume, ...",2018-03-27 21:46:10,1999-11-18,2018-03-27
1,AA,"Alcoa Inc. (AA) Prices, Dividends, Splits and ...","End of day open, high, low, close and volume, ...",2018-03-27 21:46:10,2016-11-01,2018-03-27
2,AAL,"American Airlines Group Inc. (AAL) Prices, Div...","<p>End of day open, high, low, close and volum...",2018-03-27 21:46:10,2005-09-27,2018-03-27
3,AAMC,"Altisource Asset Management (AAMC) Prices, Div...","<p>End of day open, high, low, close and volum...",2018-03-27 21:46:07,2012-12-13,2018-03-27
4,AAN,"Aaron's Inc. (AAN) Prices, Dividends, Splits a...","<p>End of day open, high, low, close and volum...",2018-03-27 21:45:53,1984-09-07,2018-03-27
5,AAOI,"Applied Optoelectronics Inc (AAOI) Prices, Div...","<p>End of day open, high, low, close and volum...",2018-03-27 21:46:07,2013-09-26,2018-03-27
6,AAON,"AAON Inc. (AAON) Prices, Dividends, Splits and...","<p>End of day open, high, low, close and volum...",2018-03-27 21:45:53,1992-12-16,2018-03-27
7,AAP,"Advance Auto Parts Inc (AAP) Prices, Dividends...","<p>End of day open, high, low, close and volum...",2018-03-27 21:45:53,2001-11-29,2018-03-27
8,AAPL,"Apple Inc (AAPL) Prices, Dividends, Splits and...","End of day open, high, low, close and volume, ...",2018-03-27 21:46:10,1980-12-12,2018-03-27
9,AAT,"American Assets Trust Inc. (AAT) Prices, Divid...","<p>End of day open, high, low, close and volum...",2018-03-27 21:46:07,2011-01-13,2018-03-27


In [72]:
dataset_df

,"{""data"":{""files"":[]",latest_full_data:null}}


* Since the api for `get`'ing seems to be not working, we can use the **quandl** module to download the dataset.



* The WIKI dataset has a huge variety of bonds, stocks and assets, at first, we'll be dealing with just **two**. The *Apple*(AAPL) and the *Tesla* (TSLA)



* Also, for the sake of implementation, only one year of data will be downloaded and loaded into memory. Later we can experiment with multiple timeframes for cross validation of models.

In [94]:
quandl.ApiConfig.api_key = api_key

dataset_df = quandl.get_table('WIKI/PRICES', ticker = ['AAPL', 'TSLA'], 
                        date = {'gte': '2015-12-31', 'lte': '2016-12-31'}, 
                        paginate=True)
dataset_df.head()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
None,,,,,,,,,,,,,,
0,TSLA,2016-12-30,216.30,217.50,211.6800,213.69,4642620.0,0.0,1.0,216.30,217.50,211.6800,213.69,4642620.0
1,TSLA,2016-12-29,218.56,219.20,214.1225,214.68,4044968.0,0.0,1.0,218.56,219.20,214.1225,214.68,4044968.0
2,TSLA,2016-12-28,221.53,223.80,217.2000,219.74,3782456.0,0.0,1.0,221.53,223.80,217.2000,219.74,3782456.0
3,TSLA,2016-12-27,214.88,222.25,214.4200,219.53,5915732.0,0.0,1.0,214.88,222.25,214.4200,219.53,5915732.0
4,TSLA,2016-12-23,208.00,213.45,207.7100,213.34,4670464.0,0.0,1.0,208.00,213.45,207.7100,213.34,4670464.0


# III) Feature engineering

* As discussed in the first section, it is useful to transform the OCHL features to something manageable mathematically, such as, a (locally) stationary time series.

### Log returns.

* Among the advantages explored in the **section I)** of using returns instead of the raw data, we can further improve the feature of time series so it displays desirable statistical properties. One usual transform is known as **Log-returns**, defined as follows:


$$log(1 + \frac{X(t) - X(t - 1)}{X(t-1)})$$


* The desirable properties of using log-returns can be summed in as being prone to follow a normal distribution and being able to "accumulate" returns over time with simple additions instead of multiplications)